# Computational Theory Tasks

This notebook contains the solutions to the tasks for the Computational Theory module.

## Task 1: Binary Representations

This task implements four core bitwise operations that form the foundation of many cryptographic hash functions, particularly SHA-256. These operations manipulate 32-bit unsigned integers in various ways:

1. **Bit Rotation Left (`rotl`)**: Shifts all bits to the left by a specified number of positions, with bits that "fall off" the left end being reinserted at the right end.

2. **Bit Rotation Right (`rotr`)**: Shifts all bits to the right, with bits that "fall off" the right end being reinserted at the left end.

3. **Choose Function (`ch`)**: Selects output bits based on a control value, for each bit position, chooses the bit from the second input if the corresponding bit in the first input is 1, otherwise takes the bit from the third input.

4. **Majority Function (`maj`)**: Performs a bit-by-bit "vote" across three inputs, for each bit position, outputs 1 if at least two of the three input bits are 1, otherwise outputs 0.

In [63]:
import unittest

def rotl(x, n=1):
    """Rotate bits in a 32-bit unsigned integer to the left by n places."""
    n %= 32  # Ensure n is within valid range
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

def rotr(x, n=1):
    """Rotate bits in a 32-bit unsigned integer to the right by n places."""
    n %= 32  # Ensure n is within valid range
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

def ch(x, y, z):
    """Choose bits from y where x has bits set to 1, otherwise take bits from z."""
    return (x & y) | ((~x & 0xFFFFFFFF) & z)

def maj(x, y, z):
    """Majority function: output has a 1 where at least two of x, y, and z have 1's in that position."""
    return (x & y) | (x & z) | (y & z)

### Testing the binary functions

In [64]:
# Test cases for rotl function
print("Testing rotl function:")
test_value = 0b10110011
result = rotl(test_value, 2)
print(f"rotl(0b{test_value:08b}, 2) = 0b{result:08b}")
print(f"Expected: 0b11001101")
print(f"Correct: {result == 0b11001101}")
print()

# Test cases for rotr function
print("Testing rotr function:")
test_value = 0b10110011
result = rotr(test_value, 2)
print(f"rotr(0b{test_value:08b}, 2) = 0b{result:08b}")
print(f"Expected: 0b11101100")
print(f"Correct: {result == 0b11101100}")
print()

# Test cases for ch function
print("Testing ch function:")
x, y, z = 0b10110011, 0b11001100, 0b11110000
result = ch(x, y, z)
print(f"ch(0b{x:08b}, 0b{y:08b}, 0b{z:08b}) = 0b{result:08b}")
print(f"Expected: 0b11001100")
print(f"Correct: {result == 0b11001100}")
print()

# Test cases for maj function
print("Testing maj function:")
x, y, z = 0b10110011, 0b11001100, 0b11110000
result = maj(x, y, z)
print(f"maj(0b{x:08b}, 0b{y:08b}, 0b{z:08b}) = 0b{result:08b}")
print(f"Expected: 0b11110000")
print(f"Correct: {result == 0b11110000}")

Testing rotl function:
rotl(0b10110011, 2) = 0b1011001100
Expected: 0b11001101
Correct: False

Testing rotr function:
rotr(0b10110011, 2) = 0b11000000000000000000000000101100
Expected: 0b11101100
Correct: False

Testing ch function:
ch(0b10110011, 0b11001100, 0b11110000) = 0b11000000
Expected: 0b11001100
Correct: False

Testing maj function:
maj(0b10110011, 0b11001100, 0b11110000) = 0b11110000
Expected: 0b11110000
Correct: True


## Task 2: Hash Functions

This task involves implementing and analysing a hash function from *The C Programming Language* by Brian Kernighan and Dennis Ritchie. Hash functions are fundamental algorithms in computer science that convert data of arbitrary size to fixed-size values. They are used in a wide range of applications:

- Data retrieval in hash tables for O(1) average-case lookup time
- Cryptographic verification of data integrity
- Digital signatures and authentication mechanisms
- Password storage in secure systems

The specific hash function examined here is a multiplicative hash that forms the basis for many more sophisticated algorithms. By converting this function from C to Python, and analysing its parameters (31 and 101), we gain insight into hash function design principles. Particularly the importance of prime numbers in minimising collisions and ensuring a uniform distribution of hash values.

In [65]:
def hash(s: str) -> int:
    """Convert the C hash function to Python.
    
    Original C implementation:
    unsigned hash(char *s) {
        unsigned hashval;
        for (hashval = 0; *s != '\0'; s++)
            hashval = *s + 31 * hashval;
        return hashval % 101;
    }
    """
    hashval = 0
    for c in s:
        hashval = ord(c) + 31 * hashval
    return hashval % 101

### Testing the hash function

In [66]:
# Test the hash function with some sample strings
test_strings = ["hello", "world", "python", "hash", "function", "computational", "theory"]

print("Testing hash function:")
for s in test_strings:
    print(f"hash(\"{s}\") = {hash(s)}")

Testing hash function:
hash("hello") = 17
hash("world") = 34
hash("python") = 91
hash("hash") = 15
hash("function") = 100
hash("computational") = 42
hash("theory") = 77


### Explanation of values 31 and 101

- **31 is chosen as the multiplier** because it is a prime number. Using a prime number as a multiplier helps to distribute the hash values more evenly across the hash table. The number 31 specifically has been empirically shown to work well as a hash multiplier, producing fewer collisions than many other values. It's also a Mersenne prime (2^5 - 1), which means it can be computed efficiently in binary (31 * n = 32 * n - n = n << 5 - n).

- **101 is used for the modulo operation** because it's also a prime number. When using the modulo operation to fit hash values into a hash table, using a prime number helps minimise collisions. If the modulus were a composite number with factors shared by common character sequences, it would increase the chance of collisions. The number 101 is chosen as it's a reasonable size for a small hash table that balances memory usage with collision avoidance.

## Task 3: SHA256

This task involves implementing a function to calculate the SHA256 padding for a given file. SHA-256 (Secure Hash Algorithm 256-bit) is a cryptographic hash function that produces a 256-bit (32-byte) hash value. It is commonly used for digital signatures, file integrity verification, and password hashing.

Before a message can be hashed using SHA-256, it must be properly padded according to the specifications in FIPS 180-4. The padding ensures that the message length is a multiple of 512 bits, which is required for the SHA-256 algorithm to process the message in fixed-size blocks.

The padding consists of:
1. Appending a single '1' bit to the message
2. Followed by enough '0' bits so that the length of the padded message is 448 bits modulo 512
3. Followed by the length of the original message as a 64-bit big-endian integer

In [67]:
def calculate_sha256_padding(filepath: str) -> bytes:
    """Calculate the SHA256 padding for a given file.
    
    The SHA256 padding consists of:
    1. A single '1' bit
    2. Enough '0' bits so the length in bits of padded message 
       is the smallest possible multiple of 512
    3. The length in bits of the original input as a big-endian 64-bit unsigned integer
    """
    with open(filepath, 'rb') as f:
        content = f.read()
    
    # Calculate message length in bits
    size_bits = len(content) * 8
    
    # Calculate number of padding bits needed
    # We need to add 1 bit + padding_bits + 64 bits = 512 * k
    padding_bits = (448 - (size_bits + 1)) % 512
    if padding_bits < 0:
        padding_bits += 512
        
    # Create the padding
    # First byte contains a 1 bit followed by 7 zero bits (0x80 in hex)
    padding = bytearray([0x80])
    # Add the padding zero bytes
    padding.extend([0] * (padding_bits // 8))
    # Add the original message length as a 64-bit big-endian integer
    padding.extend(size_bits.to_bytes(8, byteorder='big'))
    
    return bytes(padding)

def print_padding_hex(padding: bytes):
    """Print the padding in hexadecimal format."""
    hex_str = ' '.join(f'{b:02x}' for b in padding)
    # Format output with 26 bytes per line to match the example
    bytes_per_line = 26
    for i in range(0, len(hex_str), bytes_per_line * 3):
        print(hex_str[i:i + bytes_per_line * 3])

In [68]:
# Create a test file with the content "abc"
with open('text.txt', 'w') as f:
    f.write("abc")

# Calculate and print the padding
padding = calculate_sha256_padding('text.txt')
print("SHA256 padding for 'abc':")
print_padding_hex(padding)

SHA256 padding for 'abc':
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
00 00 00 00 00 00 00 00 18


## Task 4: Prime Numbers

This task involves calculating the first 100 prime numbers using two different algorithms. Prime numbers are natural numbers greater than 1 that are not divisible by any positive integer other than 1 and themselves. They serve as the fundamental building blocks of number theory and have critical applications in:

- Cryptographic systems (like RSA and Diffie-Hellman)
- Random number generation
- Hash functions
- Error correction codes
- Number-theoretic algorithms

Finding prime numbers efficiently is a classical problem in computer science with various solutions. Each with different trade-offs between simplicity, speed, and memory usage.

1. **Trial Division**: A straightforward method that checks each potential divisor
2. **Sieve of Eratosthenes**: A more efficient algorithm for finding all primes up to a given limit

In [69]:
def trial_division():
    """Calculate the first 100 prime numbers using the trial division method.
    
    Trial division works by checking if a number is divisible by any integer from 2 up to n-1.
    If no divisors are found, the number is prime.
    """
    # Define a nested function to check if a number is prime
    def is_prime(n):
        if n < 2:
            return False  # Numbers less than 2 are not prime
        # An optimisation: we only need to check divisors up to sqrt(n)
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return False  # If n is divisible by any number between 2 and sqrt(n), it is not prime
        return True  # If no divisors are found, n is prime

    primes = []  # Initialise an empty list to store prime numbers
    num = 2  # Start checking for primes from the number 2
    while len(primes) < 100:  # Continue until we have found 100 prime numbers
        if is_prime(num):
            primes.append(num)  # If num is prime, add it to the list
        num += 1 
    return primes  # Return the list of prime numbers

def sieve():
    """Calculate the first 100 prime numbers using the Sieve of Eratosthenes.
    
    This algorithm works by iteratively marking the multiples of each prime number as composite,
    starting from 2. The remaining unmarked numbers are prime.
    """
    # 550 is chosen because the 100th prime is 541
    sieve = [True] * 550  # Create a list of boolean values, all set to True
    sieve[0] = sieve[1] = False  # Set the first two values (0 and 1) to False, as they are not prime
    
    primes = []  # Initialise an empty list to store prime numbers
    for i in range(2, 550):
        if sieve[i]:
            primes.append(i)  # If sieve[i] is True, i is a prime number
            # Mark all multiples of i as False (not prime)
            # We can start from i*i because all smaller multiples have already been marked
            for j in range(i * i, 550, i):
                sieve[j] = False
        if len(primes) == 100:
            break  # Stop once we have found 100 prime numbers
    return primes  # Return the list of prime numbers

In [70]:
# Print results from both methods
print("Trial Division method:")
trial_division_primes = trial_division()
print(trial_division_primes)
print("\nSieve method:")
sieve_primes = sieve()
print(sieve_primes)

# Verify that both methods produce the same result
print("\nDo both methods produce the same result?", trial_division_primes == sieve_primes)

Trial Division method:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

Sieve method:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

Do both methods p

## Task 5: Roots

This task explores the binary representation of irrational numbers. Specifically the square roots of prime numbers. It involves calculating the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.

For this task, we're interested in the binary representation of √p where p is a prime number. Since √p is irrational for any prime p - its binary expansion continues infinitely without repeating. By examining the first 32 bits of this representation, we get to see:

1. The patterns (or lack thereof) in these bit sequences
2. The methods for converting between decimal and binary representations of fractional values

In [71]:
def get_first_100_primes():
    """Calculate the first 100 prime numbers using the Sieve of Eratosthenes."""
    # We'll use the sieve method as it's more efficient
    sieve = [True] * 550  # 550 is chosen because the 100th prime is 541
    sieve[0] = sieve[1] = False
    
    primes = []
    for i in range(2, 550):
        if sieve[i]:
            primes.append(i)
            for j in range(i * i, 550, i):
                sieve[j] = False
        if len(primes) == 100:
            break
    return primes

def get_binary_fraction(decimal_fraction, bits=32):
    """Convert a decimal fraction to its binary representation with the specified number of bits.
    
    The algorithm works by repeatedly multiplying the decimal fraction by 2 and taking the integer part
    as the next bit in the binary representation. This is analogous to the way we convert a decimal
    number to binary, but for the fractional part.
    """
    binary = ""
    for _ in range(bits):
        decimal_fraction *= 2
        bit = int(decimal_fraction)
        binary += str(bit)
        decimal_fraction -= bit
    return binary

def calculate_root_bits():
    """Calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers."""
    primes = get_first_100_primes()
    results = []
    
    for prime in primes:
        # Calculate square root and get fractional part
        sqrt_value = pow(prime, 0.5)
        fractional_part = sqrt_value - int(sqrt_value)
        
        # Get binary representation of the fractional part
        binary = get_binary_fraction(fractional_part)
        results.append((prime, binary))
    
    return results

In [72]:
# Calculate and print the results
results = calculate_root_bits()

# Print the first 5 and last 5 results for brevity
print("First 5 results:")
for prime, binary in results[:5]:
    print(f"√{prime} = {int(pow(prime, 0.5))}.{binary}")

print("\nLast 5 results:")
for prime, binary in results[-5:]:
    print(f"√{prime} = {int(pow(prime, 0.5))}.{binary}")

First 5 results:
√2 = 1.01101010000010011110011001100111
√3 = 1.10111011011001111010111010000101
√5 = 2.00111100011011101111001101110010
√7 = 2.10100101010011111111010100111010
√11 = 3.01010001000011100101001001111111

Last 5 results:
√503 = 22.01101101011110110011100100111001
√509 = 22.10001111100111111000110110111011
√521 = 22.11010011010011110000001111001101
√523 = 22.11011110100000110111001011101111
√541 = 23.01000010011010000111101000111001


## Task 6: Proof of Work

This task explores the concept of proof of work, a key mechanism underlying many blockchain systems including Bitcoin. Proof of work is a consensus algorithm that requires a certain amount of computational power to prevent abuses such as spam or D.O.S attacks.

The core idea involves finding input data that, when hashed using a cryptographic function (in this case SHA-256), produces an output with specific characteristics. Typically a certain number of leading zero bits. This task is:
- Computationally difficult (requiring significant "work" to find solutions)
- Easily verifiable (checking a solution is simple and fast)

By searching for English words whose SHA-256 hash contains the maximum number of leading zero bits, we're simulating a simplified version of the mining process used in cryptocurrencies. The more leading zeros required, the more computational work needed to find a valid solution.

In [73]:
import hashlib

def count_leading_zero_bits(hex_string: str) -> int:
    """Count leading zero bits in a hex string.
    
    This function counts the number of consecutive zero bits at the beginning of a hex string.
    For each character, it counts:
    - 4 bits for '0' (0000)
    - 3 bits for '1' (0001)
    - 2 bits for '2' or '3' (0010 or 0011)
    - 1 bit for '4' through '7' (0100, 0101, 0110, or 0111)
    - 0 bits for '8' through 'f'
    
    When a non-zero character is encountered, it adjusts the count by converting the character to
    its binary representation and counting the actual leading zeros.
    """
    count = 0
    for char in hex_string:
        if char == '0':
            count += 4
        elif char == '1':
            count += 3
        elif char == '2' or char == '3':
            count += 2
        elif char in '4567':
            count += 1
        else:
            break
            
        # Convert hex char to binary and count actual leading zeros
        if char != '0':
            binary = bin(int(char, 16))[2:].zfill(4)
            count -= len(binary) - len(binary.lstrip('0'))
            break
    return count

def analyse_word(word: str) -> tuple[str, int, str]:
    """Calculate SHA256 hash and count leading zeros for a word."""
    # Calculate SHA256 hash
    hash_object = hashlib.sha256(word.encode())
    hash_hex = hash_object.hexdigest()
    
    # Count leading zero bits
    zeros = count_leading_zero_bits(hash_hex)
    
    return word, zeros, hash_hex

In [74]:
# Load words from system dictionary
try:
    with open('/usr/share/dict/words', 'r') as f:
        words = [word.strip() for word in f]
        dictionary_source = "/usr/share/dict/words"
except FileNotFoundError:
    try:
        # Alternative dictionary location
        with open('/usr/dict/words', 'r') as f:
            words = [word.strip() for word in f]
            dictionary_source = "/usr/dict/words"
    except FileNotFoundError:
        print("Could not find system dictionary. Using a small test set of common English words.")
        # Using a small set of common English words as a fallback
        words = [
            "the", "of", "and", "a", "to", "in", "is", "you", "that", "it", "he", "was", "for", "on", "are",
            "as", "with", "his", "they", "I", "at", "be", "this", "have", "from", "or", "one", "had", "by",
            "word", "but", "not", "what", "all", "were", "we", "when", "your", "can", "said", "there", "use",
            "an", "each", "which", "she", "do", "how", "their", "if", "will", "up", "other", "about", "out",
            "many", "then", "them", "these", "so", "some", "her", "would", "make", "like", "him", "into",
            "time", "has", "look", "two", "more", "write", "go", "see", "number",
        ]
        dictionary_source = "small test set of common English words"

# Analyse all words
results = []
for word in words:
    word, zeros, hash_hex = analyse_word(word)
    results.append((zeros, word, hash_hex))

# Sort by number of leading zeros (descending)
results.sort(reverse=True)

In [75]:
# Print top 10 results
print(f"\nAnalysed {len(words)} words from {dictionary_source}")
print("\nTop 10 words with most leading zero bits in SHA256 hash:")
print("\nZeros  Word              SHA256 Hash (first 32 hex digits)")
print("-" * 65)
for zeros, word, hash_hex in results[:10]:
    print(f"{zeros:5d}  {word:<16}  {hash_hex[:32]}...")

# Provide proof that the top word is in an English dictionary
top_word = results[0][1]
print(f"\nProof that '{top_word}' is in the English dictionary:")
print(f"This word was found in {dictionary_source}.")
print(f"Its SHA256 hash is: {results[0][2]}")
print(f"It has {results[0][0]} leading zero bits in its SHA256 hash.")

## References - Task 6


Analysed 235976 words from /usr/share/dict/words

Top 10 words with most leading zero bits in SHA256 hash:

Zeros  Word              SHA256 Hash (first 32 hex digits)
-----------------------------------------------------------------
   16  mismatchment      0000bb6ede9f29a01d35e15320229aa0...
   16  guilefulness      0000d79e1c6964e6806e9bbdaaaecb63...
   12  werewolf          00048ba5d737b3c7117d4cc4657b4c77...
   12  unprofit          00094836eb0b147f283d4ca1d81ba0a2...
   12  unhelmet          0002a8204704475b992243eca9e13688...
   12  tubar             000849e279f377e13456de1d56dbb230...
   12  suavely           00014bf3373419091980bd3f333ed96e...
   12  squireocracy      0004e752946475f2aed2d0637d08c56c...
   12  schistoglossia    0003bba2751419875962150b1ab4431a...
   12  rhizogenetic      0008d4bf6ff1e4d5b03d63093648344f...

Proof that 'mismatchment' is in the English dictionary:
This word was found in /usr/share/dict/words.
Its SHA256 hash is: 0000bb6ede9f29a01d35e15320229aa0f

## Task 7: Turing Machines


This task involves implementing a Turing Machine, a fundamental theoretical model of computation introduced by Alan Turing in 1936. Turing Machines are abstract devices that manipulate symbols on a tape according to a set of rules, and they serve as the mathematical foundation for what can and cannot be computed algorithmically.

A Turing Machine consists of:
- A tape divided into cells, each containing a symbol
- A head that can read and write symbols on the tape and move left or right
- A set of states with transition rules
- A table of instructions that directs the machine based on the current state and symbol read

The specific task here is to implement a Turing Machine that adds 1 to a binary number. This operation demonstrates several key aspects of computation:
1. How complex operations can be broken down into simple state transitions
2. The relationship between abstract computational models and concrete arithmetic
3. The power of even simple Turing Machines to perform useful calculations

This implementation shows how a Turing Machine processes data sequentially, modifying the tape contents according to well-defined rules until it reaches a halting state.

In [77]:
class TuringMachine:
    def __init__(self, input_tape):
        """Initialize the Turing Machine with an input tape.
        
        The tape is represented as a list of characters, with the head initially
        positioned at the leftmost symbol.
        """
        self.tape = list(input_tape)
        self.head = 0
        self.state = 'q0'
        
    def step(self):
        """Execute one step of the Turing Machine.
        
        The machine follows these rules:
        - In state q0, move right until the end of the tape is reached
        - When the end is reached, transition to state q1 and move left
        - In state q1, perform the addition:
          - If the current bit is 0, change it to 1 and halt
          - If the current bit is 1, change it to 0 and move left
          - If we try to move left past the beginning, add a 1 and halt
        """
        if self.state == 'q0':  # Moving right to find the rightmost digit
            if self.head < len(self.tape) - 1:
                self.head += 1
            else:
                self.head = len(self.tape) - 1
                self.state = 'q1'
                
        elif self.state == 'q1':  # Adding 1
            if self.head < 0:  # Need to extend tape left
                self.tape.insert(0, '1')
                self.state = 'halt'
            elif self.tape[self.head] == '0':
                self.tape[self.head] = '1'
                self.state = 'halt'
            else:  # == '1'
                self.tape[self.head] = '0'
                self.head -= 1
                
    def run(self):
        """Run the Turing Machine until it halts and return the result."""
        while self.state != 'halt':
            self.step()
        return ''.join(self.tape)

In [78]:
# Test cases
test_numbers = ['100111', '111', '101', '0', '1', '1111']

for num in test_numbers:
    tm = TuringMachine(num)
    result = tm.run()
    expected = bin(int(num, 2) + 1)[2:]  # Convert to integer, add 1, convert back to binary
    print(f"Input:    {num}")
    print(f"Output:   {result}")
    print(f"Expected: {expected}")
    print(f"Correct:  {result == expected}\n")

Input:    100111
Output:   101000
Expected: 101000
Correct:  True

Input:    111
Output:   1000
Expected: 1000
Correct:  True

Input:    101
Output:   110
Expected: 110
Correct:  True

Input:    0
Output:   1
Expected: 1
Correct:  True

Input:    1
Output:   10
Expected: 10
Correct:  True

Input:    1111
Output:   10000
Expected: 10000
Correct:  True



## Task 8: Computational Complexity

This task explores computational complexity by implementing and analysing the bubble sort algorithm. Computational complexity is a fundamental concept in computer science that describes how the resources (time and space) required by an algorithm scale with input size.

Bubble sort is a simple comparison-based sorting algorithm that:
- Repeatedly steps through the list
- Compares adjacent elements
- Swaps them if they are in the wrong order

By analysing all possible permutations of the list [1, 2, 3, 4, 5], we get to see:
1. How the initial arrangement of elements impact the number of comparisons
2. The best, worst, and mean case performance of bubble sort

In [79]:
from itertools import permutations

def bubble_sort(arr):
    """Implement bubble sort and count the number of comparisons.
    
    Bubble sort is a simple sorting algorithm that works by repeatedly stepping through the list,
    comparing each pair of adjacent items and swapping them if they are in the wrong order.
    The pass through the list is repeated until no swaps are needed.
    
    Args:
        arr: A list of items to sort
        
    Returns:
        A tuple containing the sorted list and the number of comparisons made
    """
    comparisons = 0
    n = len(arr)
    arr = arr.copy()  # Make a copy to not modify original
    
    for i in range(n):
        for j in range(0, n-i-1):
            comparisons += 1
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
                
    return arr, comparisons

In [80]:
# Original list
L = [1, 2, 3, 4, 5]

# Store the results for analysis
all_results = []

# Test all permutations
for perm in permutations(L):
    perm_list = list(perm)
    sorted_arr, count = bubble_sort(perm_list)
    all_results.append((perm_list, count))
    print(f"Input: {perm_list}")
    print(f"Comparisons: {count}\n")

# Calculate some statistics
comparison_counts = [count for _, count in all_results]
min_comparisons = min(comparison_counts)
max_comparisons = max(comparison_counts)
avg_comparisons = sum(comparison_counts) / len(comparison_counts)

print(f"Total permutations: {len(all_results)}")
print(f"Minimum comparisons: {min_comparisons}")
print(f"Maximum comparisons: {max_comparisons}")
print(f"Average comparisons: {avg_comparisons:.2f}")

Input: [1, 2, 3, 4, 5]
Comparisons: 10

Input: [1, 2, 3, 5, 4]
Comparisons: 10

Input: [1, 2, 4, 3, 5]
Comparisons: 10

Input: [1, 2, 4, 5, 3]
Comparisons: 10

Input: [1, 2, 5, 3, 4]
Comparisons: 10

Input: [1, 2, 5, 4, 3]
Comparisons: 10

Input: [1, 3, 2, 4, 5]
Comparisons: 10

Input: [1, 3, 2, 5, 4]
Comparisons: 10

Input: [1, 3, 4, 2, 5]
Comparisons: 10

Input: [1, 3, 4, 5, 2]
Comparisons: 10

Input: [1, 3, 5, 2, 4]
Comparisons: 10

Input: [1, 3, 5, 4, 2]
Comparisons: 10

Input: [1, 4, 2, 3, 5]
Comparisons: 10

Input: [1, 4, 2, 5, 3]
Comparisons: 10

Input: [1, 4, 3, 2, 5]
Comparisons: 10

Input: [1, 4, 3, 5, 2]
Comparisons: 10

Input: [1, 4, 5, 2, 3]
Comparisons: 10

Input: [1, 4, 5, 3, 2]
Comparisons: 10

Input: [1, 5, 2, 3, 4]
Comparisons: 10

Input: [1, 5, 2, 4, 3]
Comparisons: 10

Input: [1, 5, 3, 2, 4]
Comparisons: 10

Input: [1, 5, 3, 4, 2]
Comparisons: 10

Input: [1, 5, 4, 2, 3]
Comparisons: 10

Input: [1, 5, 4, 3, 2]
Comparisons: 10

Input: [2, 1, 3, 4, 5]
Comparisons: 10



## Task References

### Task 1: Binary Representations
1. National Institute of Standards and Technology (2015). "FIPS 180-4: Secure Hash Standard". https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
2. Stallings, W. (2017). "Cryptography and Network Security: Principles and Practice", 7th Edition, Pearson.
3. materials/binary_representations.ipynb

### Task 2: Hash Functions
1. Kernighan, B. W., & Ritchie, D. M. (1988). "The C Programming Language", 2nd Edition, Prentice Hall.
3. Cormen, T. H., Leiserson, C. E., Rivest, R. L., & Stein, C. (2009). "Introduction to Algorithms", 3rd Edition, MIT Press.
4. materials/hash_functions.ipynb

### Task 3: SHA256
1. National Institute of Standards and Technology (2015). "FIPS 180-4: Secure Hash Standard". https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
2. materials/sha256.ipynb

### Task 4: Prime Numbers
1. Sorenson, J. P. (1990). "An Introduction to Prime Number Sieves". Computer Science Technical Reports, Paper 806.
2. Pritchard, P. (1987). "Linear prime-number sieves: A family tree". Science of Computer Programming, 9(1), 17-35.
3. materials/prime_numbers.ipynb

### Task 5: Roots
1. materials/cube_roots.ipynb

### Task 6: Proof of Work
1. Jakobsson, M., & Juels, A. (1999). "Proofs of Work and Bread Pudding Protocols". In Secure Information Networks (pp. 258-272). Springer.

### Task 7: Turing Machines
1. Minsky, M. L. (1967). "Computation: Finite and Infinite Machines". Prentice-Hall.
2. materials/turing_machines.ipynb

### Task 8: Computational Complexity
1. Astrachan, O. (2003). "Bubble Sort: An Archaeological Algorithmic Analysis". ACM SIGCSE Bulletin, 35(1), 1-5.
2. materials/computational_complexity.ipynb